In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [7]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
niter = 500
rank = 80
tau = 3
gamma = 0.5
rho = 0.5
alpha = 1



print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
    d0 = 561 #561 =3*11*17

    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 512
    d6 = 6 


    W1 = 0.2*init.xavier_normal_(torch.empty(d1, d0, device=device), gain=1.0)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.2*init.xavier_normal_(torch.empty(d2, d1, device=device), gain=1.0)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.2*init.xavier_normal_(torch.empty(d3, d2, device=device), gain=1.0)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.2*init.xavier_normal_(torch.empty(d4, d3, device=device), gain=1.0)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)

    W5 = 0.2*init.xavier_normal_(torch.empty(d5, d4, device=device), gain=1.0)
    W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())
    factors5 = tensor_train(W5_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W5_tl_tensor_rec = tt_to_tensor(factors5)
    b5 = 0*torch.ones(d5, 1, device=device)


    W6 = 0.2*init.xavier_normal_(torch.empty(d6, d5, device=device), gain=1.0)
    b6 = 0*torch.ones(d6, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = nn.ReLU()(U5)
    U6 = torch.addmm(b6.repeat(1, N), W6, V5)
    V6 = U6 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V6 = (y_one_hot + gamma*U6 + alpha*V6)/(1 + gamma + alpha)

        # update U4 
        U6 = (gamma*V6 + rho*(torch.mm(W6,V5) + b6.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W6, b6 = updateWb_org(U6,V5,W6,b6,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


        # update for 5th layer
        # update V3
        V5 = updateV(U5,U6,W6,b6,rho,gamma)

        # update U3
        U5 = relu_prox(V5,(rho*torch.addmm(b5.repeat(1,N), W5, V4) + alpha*U5)/(rho + alpha),(rho + alpha)/gamma,d5,N)

        # update W3 and b3
        W5, b5 = updateWb(U5,V4,W5,b5,W5_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4))
        W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors5 = tensor_train(W5_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W5_tl_tensor_rec = tt_to_tensor(factors5)


  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        a5_train = nn.ReLU()(torch.addmm(b5.repeat(1, N), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b6.repeat(1, N), W6, a5_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        a5_test = nn.ReLU()(torch.addmm(b5.repeat(1, N_test), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_test))
        pred_test = torch.argmax(torch.addmm(b6.repeat(1, N_test), W6, a5_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V6,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b6.repeat(1,N), W6, V5),U6,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,nn.ReLU()(U5),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V6,U6,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W5.reshape((4,4,4,4,4,4,4,4,4)),torch.as_tensor(W5_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))
    factors5_shape=[f.shape for f in factors5]
    Sum_of_variables_factors5=sum(list(x*y*z for x,y,z in factors5_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4+Sum_of_variables_factors5

    CR_1=((total_variabels)+(d5*d6))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5+d5*d6)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#     #this postion to add new row into existing table
#         df=pd.read_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
#         new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
#                    'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
#                    'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1],'seed':seed} 
#         df=df.append(new_row,ignore_index=True)
#         df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv',index=False)


filename= "XavierNormal_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/5 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 3 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 1.7864327430725098 - sq_loss: 661.6837768554688 - tot_loss: 962.8837831819328 - acc: 0.15832426550598477 - val_acc: 0.15575161180861893
Repeatition 1 Epoch 2 / 500 
 - time: 1.4745683670043945 - sq_loss: 294.0816650390625 - tot_loss: 484.6620297046611 - acc: 0.17519042437431992 - val_acc: 0.167288768238887
Repeatition 1 Epoch 3 / 500 
 - time: 1.4563641548156738 - sq_loss: 163.61614990234375 - tot_loss: 264.0111472378485 - acc: 0.17437431991294886 - val_acc: 0.16389548693586697
Repeatition 1 Epoch 4 / 500 
 - time: 1.475423812866211 - sq_loss: 89.1167984008789 - tot_loss: 148.24794450122863 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 5 / 500 
 - time: 1.4267325401306152 - sq_loss: 48.02313232421875 - tot_loss: 85.99057460390031 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 6 / 500 
 - time: 1.44

Repeatition 1 Epoch 47 / 500 
 - time: 1.4306671619415283 - sq_loss: 0.0004327986389398575 - tot_loss: 1.3466166389644059 - acc: 0.5442056583242655 - val_acc: 0.5296912114014252
Repeatition 1 Epoch 48 / 500 
 - time: 1.4392459392547607 - sq_loss: 0.0004039535124320537 - tot_loss: 1.3157404584544565 - acc: 0.5458378672470077 - val_acc: 0.5310485239226331
Repeatition 1 Epoch 49 / 500 
 - time: 1.4307119846343994 - sq_loss: 0.00037736172089353204 - tot_loss: 1.2506423972372431 - acc: 0.5485582154515778 - val_acc: 0.5351204614862572
Repeatition 1 Epoch 50 / 500 
 - time: 1.435593605041504 - sq_loss: 0.0003530910180415958 - tot_loss: 1.2224871171983978 - acc: 0.5572633297062024 - val_acc: 0.5391923990498813
Repeatition 1 Epoch 51 / 500 
 - time: 1.436711072921753 - sq_loss: 0.00033075339160859585 - tot_loss: 1.1783856397687487 - acc: 0.5663764961915125 - val_acc: 0.5449609772650152
Repeatition 1 Epoch 52 / 500 
 - time: 1.435814619064331 - sq_loss: 0.0003101137699559331 - tot_loss: 1.147059

Repeatition 1 Epoch 94 / 500 
 - time: 2.825350761413574 - sq_loss: 2.9215418180683628e-05 - tot_loss: 0.3554635995232047 - acc: 0.8669749727965179 - val_acc: 0.844248388191381
Repeatition 1 Epoch 95 / 500 
 - time: 2.837512254714966 - sq_loss: 2.7683208827511407e-05 - tot_loss: 0.3382630403662006 - acc: 0.8683351468988031 - val_acc: 0.8469630132337971
Repeatition 1 Epoch 96 / 500 
 - time: 2.7913858890533447 - sq_loss: 2.6269110094290227e-05 - tot_loss: 0.3336273159920893 - acc: 0.8710554951033732 - val_acc: 0.8506956226671191
Repeatition 1 Epoch 97 / 500 
 - time: 2.8108878135681152 - sq_loss: 2.4928738639573567e-05 - tot_loss: 0.3192149814467484 - acc: 0.8730957562568009 - val_acc: 0.8530709195792331
Repeatition 1 Epoch 98 / 500 
 - time: 2.860074281692505 - sq_loss: 2.3664451873628423e-05 - tot_loss: 0.33846469994830386 - acc: 0.874727965179543 - val_acc: 0.8557855446216491
Repeatition 1 Epoch 99 / 500 
 - time: 2.8017325401306152 - sq_loss: 2.244572533527389e-05 - tot_loss: 0.3246

Repeatition 1 Epoch 140 / 500 
 - time: 2.740168333053589 - sq_loss: 3.573416506696958e-06 - tot_loss: 0.1899405374170584 - acc: 0.9079162132752993 - val_acc: 0.9022734984730234
Repeatition 1 Epoch 141 / 500 
 - time: 2.788569927215576 - sq_loss: 3.4525526189099764e-06 - tot_loss: 0.18874734358691114 - acc: 0.9092763873775843 - val_acc: 0.9039701391245334
Repeatition 1 Epoch 142 / 500 
 - time: 2.863085985183716 - sq_loss: 3.3375613384123426e-06 - tot_loss: 0.19209755232334658 - acc: 0.9095484221980413 - val_acc: 0.9043094672548354
Repeatition 1 Epoch 143 / 500 
 - time: 2.790290594100952 - sq_loss: 3.2277910122502362e-06 - tot_loss: 0.18288974894292664 - acc: 0.9102285092491839 - val_acc: 0.9056667797760435
Repeatition 1 Epoch 144 / 500 
 - time: 2.8723511695861816 - sq_loss: 3.129454853478819e-06 - tot_loss: 0.1793075008110634 - acc: 0.9111806311207835 - val_acc: 0.9060061079063454
Repeatition 1 Epoch 145 / 500 
 - time: 2.792630195617676 - sq_loss: 3.032100948985317e-06 - tot_loss: 

Repeatition 1 Epoch 186 / 500 
 - time: 2.7990896701812744 - sq_loss: 1.3454001646096003e-06 - tot_loss: 0.14698265619294126 - acc: 0.9328073993471164 - val_acc: 0.9300984051577876
Repeatition 1 Epoch 187 / 500 
 - time: 2.8376824855804443 - sq_loss: 1.3315476508068969e-06 - tot_loss: 0.14875257143788367 - acc: 0.9329434167573449 - val_acc: 0.9304377332880895
Repeatition 1 Epoch 188 / 500 
 - time: 2.8071343898773193 - sq_loss: 1.3154244697943795e-06 - tot_loss: 0.141346858674245 - acc: 0.9336235038084875 - val_acc: 0.9314557176789956
Repeatition 1 Epoch 189 / 500 
 - time: 2.8676962852478027 - sq_loss: 1.3017019000471919e-06 - tot_loss: 0.13641432772948825 - acc: 0.933759521218716 - val_acc: 0.9314557176789956
Repeatition 1 Epoch 190 / 500 
 - time: 2.766308069229126 - sq_loss: 1.2915122624690412e-06 - tot_loss: 0.14909330098770113 - acc: 0.9341675734494015 - val_acc: 0.9314557176789956
Repeatition 1 Epoch 191 / 500 
 - time: 2.801002264022827 - sq_loss: 1.2822350754504441e-06 - tot_l

...

Repeatition 1 Epoch 278 / 500 
 - time: 2.881479501724243 - sq_loss: 8.720475648260617e-07 - tot_loss: 0.11893827639409782 - acc: 0.9624591947769314 - val_acc: 0.9501187648456056
Repeatition 1 Epoch 279 / 500 
 - time: 2.7947535514831543 - sq_loss: 8.689273727213731e-07 - tot_loss: 0.11861059223789194 - acc: 0.9624591947769314 - val_acc: 0.9501187648456056
Repeatition 1 Epoch 280 / 500 
 - time: 2.861276865005493 - sq_loss: 8.658500405545055e-07 - tot_loss: 0.13427761827259221 - acc: 0.9630032644178455 - val_acc: 0.9504580929759077
Repeatition 1 Epoch 281 / 500 
 - time: 2.8675460815429688 - sq_loss: 8.625997338640445e-07 - tot_loss: 0.13140704116164148 - acc: 0.9630032644178455 - val_acc: 0.9504580929759077
Repeatition 1 Epoch 282 / 500 
 - time: 2.8486738204956055 - sq_loss: 8.598208864896151e-07 - tot_loss: 0.12381409030583201 - acc: 0.9630032644178455 - val_acc: 0.9501187648456056
Repeatition 1 Epoch 283 / 500 
 - time: 2.8561203479766846 - sq_loss: 8.591239861743816e-07 - tot_loss

Repeatition 1 Epoch 324 / 500 
 - time: 2.7957262992858887 - sq_loss: 7.830225285943015e-07 - tot_loss: 0.1201757734137181 - acc: 0.9704842219804135 - val_acc: 0.9565659993213438
Repeatition 1 Epoch 325 / 500 
 - time: 2.802046775817871 - sq_loss: 7.81736332555738e-07 - tot_loss: 0.11006599333342337 - acc: 0.970620239390642 - val_acc: 0.9562266711910418
Repeatition 1 Epoch 326 / 500 
 - time: 2.8201208114624023 - sq_loss: 7.802509003340674e-07 - tot_loss: 0.11485639354636912 - acc: 0.970620239390642 - val_acc: 0.9562266711910418
Repeatition 1 Epoch 327 / 500 
 - time: 2.7785842418670654 - sq_loss: 7.787100457790075e-07 - tot_loss: 0.12439034431544749 - acc: 0.970620239390642 - val_acc: 0.9562266711910418
Repeatition 1 Epoch 328 / 500 
 - time: 2.757631301879883 - sq_loss: 7.770178740429401e-07 - tot_loss: 0.13568859502462538 - acc: 0.970620239390642 - val_acc: 0.9565659993213438
Repeatition 1 Epoch 329 / 500 
 - time: 2.8012609481811523 - sq_loss: 7.753201316518243e-07 - tot_loss: 0.12

Repeatition 1 Epoch 370 / 500 
 - time: 2.8457400798797607 - sq_loss: 7.148644272092497e-07 - tot_loss: 0.12059567734834942 - acc: 0.9749727965179543 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 371 / 500 
 - time: 2.7930305004119873 - sq_loss: 7.142519962144434e-07 - tot_loss: 0.11505840302291404 - acc: 0.9752448313384113 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 372 / 500 
 - time: 2.7645721435546875 - sq_loss: 7.133117492230667e-07 - tot_loss: 0.10530058033119949 - acc: 0.9751088139281828 - val_acc: 0.9582626399728538
Repeatition 1 Epoch 373 / 500 
 - time: 2.795748710632324 - sq_loss: 7.119061251614767e-07 - tot_loss: 0.12495668361345214 - acc: 0.9752448313384113 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 374 / 500 
 - time: 2.839797258377075 - sq_loss: 7.101843948476017e-07 - tot_loss: 0.12159652050619396 - acc: 0.9752448313384113 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 375 / 500 
 - time: 2.8371849060058594 - sq_loss: 7.092499458849488e-07 - tot_loss

Repeatition 1 Epoch 416 / 500 
 - time: 2.8026535511016846 - sq_loss: 6.617320877921884e-07 - tot_loss: 0.1323429784639628 - acc: 0.9775571273122959 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 417 / 500 
 - time: 2.806072235107422 - sq_loss: 6.606380225093744e-07 - tot_loss: 0.12497473651903401 - acc: 0.9774211099020674 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 418 / 500 
 - time: 2.7736334800720215 - sq_loss: 6.595542458853743e-07 - tot_loss: 0.1197325095325239 - acc: 0.9775571273122959 - val_acc: 0.9582626399728538
Repeatition 1 Epoch 419 / 500 
 - time: 2.810361385345459 - sq_loss: 6.59085117149516e-07 - tot_loss: 0.11702912286139366 - acc: 0.9779651795429815 - val_acc: 0.9582626399728538
Repeatition 1 Epoch 420 / 500 
 - time: 2.84899640083313 - sq_loss: 6.580361286978587e-07 - tot_loss: 0.131829891751846 - acc: 0.9782372143634385 - val_acc: 0.9582626399728538
Repeatition 1 Epoch 421 / 500 
 - time: 2.778761386871338 - sq_loss: 6.569430865965842e-07 - tot_loss: 0.1296

Repeatition 1 Epoch 462 / 500 
 - time: 2.8360228538513184 - sq_loss: 6.160705652291654e-07 - tot_loss: 0.11805322093093695 - acc: 0.9795973884657236 - val_acc: 0.9582626399728538
Repeatition 1 Epoch 463 / 500 
 - time: 2.7658307552337646 - sq_loss: 6.154453444651153e-07 - tot_loss: 0.1126662752142239 - acc: 0.9795973884657236 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 464 / 500 
 - time: 2.774756908416748 - sq_loss: 6.141049198049586e-07 - tot_loss: 0.13053010326038195 - acc: 0.9795973884657236 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 465 / 500 
 - time: 2.849656820297241 - sq_loss: 6.130237579782261e-07 - tot_loss: 0.12258698658455447 - acc: 0.9795973884657236 - val_acc: 0.9586019681031558
Repeatition 1 Epoch 466 / 500 
 - time: 2.821397066116333 - sq_loss: 6.119250315350655e-07 - tot_loss: 0.11319078028261942 - acc: 0.9793253536452666 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 467 / 500 
 - time: 2.839071035385132 - sq_loss: 6.112449568718148e-07 - tot_loss: 0

In [8]:
seed

10